In [1]:
 ! wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz

--2020-08-03 07:48:48--  https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.163.21
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.163.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1556914727 (1.4G) [application/x-tar]
Saving to: ‘imagenette2.tgz’

imagenette2.tgz     100%[===================>]   1.45G  71.8MB/s    in 23s     

2020-08-03 07:49:11 (64.9 MB/s) - ‘imagenette2.tgz’ saved [1556914727/1556914727]



In [2]:
! tar -xf imagenette2.tgz

In [3]:
imagenette_map = { 
    "n01440764" : "tench",
    "n02102040" : "springer",
    "n02979186" : "casette_player",
    "n03000684" : "chain_saw",
    "n03028079" : "church",
    "n03394916" : "French_horn",
    "n03417042" : "garbage_truck",
    "n03425413" : "gas_pump",
    "n03445777" : "golf_ball",
    "n03888257" : "parachute"
}

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, InputLayer
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [5]:
imagegen = ImageDataGenerator()
train = imagegen.flow_from_directory("imagenette2/train/", class_mode = "categorical", shuffle=False, batch_size = 128, target_size=(224, 224))
val = imagegen.flow_from_directory("imagenette2/val/", class_mode = "categorical", shuffle=False, batch_size = 128, target_size=(224, 224))


Found 9469 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.


In [6]:
model = Sequential()
model.add(InputLayer(input_shape=(224, 224, 3)))

In [7]:
# 1st conv block
model.add(Conv2D(25, kernel_size=(5, 5), activation='relu', strides=(1, 1), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
# 2nd conv block
model.add(Conv2D(50, kernel_size=(5, 5), activation='relu', strides=(2, 2), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model.add(BatchNormalization())
# 3rd conv block
model.add(Conv2D(70, kernel_size=(3, 3), activation='relu', strides=(2, 2), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='valid'))
model.add(BatchNormalization())
# ANN block
model.add(Flatten())
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.25))
# output layer
model.add(Dense(units=10, activation='softmax'))

In [8]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [ ]:
model.fit_generator(train, epochs=30, validation_data=val)

KeyboardInterrupt: ignored

In [9]:
from keras.applications import VGG16

In [12]:
pretrained_model = VGG16(include_top=False, weights='imagenet')
pretrained_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [13]:
vgg_features_train = pretrained_model.predict(train)


array([0, 0, 0, ..., 9, 9, 9], dtype=int32)

In [20]:
vgg_features_val = pretrained_model.predict(val)

In [22]:
train_target = to_categorical(train.labels)
val_target = to_categorical(val.labels)

In [23]:
model2 = Sequential()
model2.add(Flatten(input_shape=(7,7,512)))
model2.add(Dense(100, activation='relu'))
model2.add(Dropout(0.5))
model2.add(BatchNormalization())
model2.add(Dense(10, activation='softmax'))

# compile the model
model2.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

model2.summary()

# train model using features generated from VGG16 model
model2.fit(vgg_features_train, train_target, epochs=50, batch_size=128, validation_data=(vgg_features_val, val_target))


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               2508900   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1010      
Total params: 2,510,310
Trainable params: 2,510,110
Non-trainable params: 200
_________________________________________________________________
Epoch 1/50
74/74 [==============================] - 5s 62ms/step - loss: 0.4456 - accuracy: 0.8843 - val_los

In [24]:
model2.save('iamgenette.h5')

In [25]:
model2.save('imagenette')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: imagenette/assets
